In [ ]:
# mv this notebook in src folder
# or
# set sys path to src folder
import os, sys
path_to_scr_folder=os.path.join(os.path.dirname(os.path.abspath('')), 'src')
sys.path.append(path_to_scr_folder)

In [ ]:
from pixel_alignment import pixel_alignment
import read_chroma
import discriminant_pixel
import baseline_correction
import find_biom
import plot
import numpy as np
from sklearn.preprocessing import normalize

In [ ]:
# labels of the chromatograms
labels = ['positif faible', 'positif', 'negatif', 'negatif', 'negatif', 'positif', 'negatif', 'positif faible', 'positif faible', 'negatif']
labels = np.array(labels)

In [ ]:
# align chromatogram
PATH_TO_THE_COHORT=...
pixel_alignment(PATH_TO_THE_COHORT)

In [ ]:
# PATH to the aligned chromatograms
ALIGNED_CHROMA_PATH = ...
aligned_chroma_files = os.listdir(ALIGNED_CHROMA_PATH)

In [ ]:
# Read aligned chromatograms and correct backgrounds
aligned_chromatos = discriminant_pixel.read_aligned_chromatos(ALIGNED_CHROMA_PATH)
aligned_chromatos = np.array([baseline_correction.chromato_no_baseline(aligned_chromato) for aligned_chromato in aligned_chromatos])

In [ ]:
# FILENAME of the chromatogram as reference. The chromatogram from which other chromatograms are aligned. Useful for relating pixels to mass spectra to indentify compounds corresponding to discriminating pixels.
ref_chromato_filename=...
chromato, time_rn, chromato_cube, sigma, mass_range=read_chroma.read_chromato_and_chromato_cube(ref_chromato_filename, mod_time=1.25, pre_process=True)

Positif (+ Positif faible) vs Negatif pixels

In [ ]:
# compute vij : assign 1 for groupe 1 labels (positif + positif faible) and -1 to others (negatif)
vij_posi_vs_other = np.ones(labels.shape)
vij_posi_vs_other[labels == 'negatif'] = -1
vij_posi_vs_other

In [ ]:
# compute discriminant compounds
matches_pos_plus_pos_f = discriminant_pixel.find_discriminant_compounds((chromato, time_rn, None), aligned_chromatos, chromato_cube, vij_posi_vs_other, disp=True, match_factor_min=500, max_pixel=500, local_max_filter=True, mod_time=1.25, title="positif+faible_")

In [ ]:
N=20
# print the N most discriminating compounds
for match in matches_pos_plus_pos_f[:N]:
    print("\"" + match[1]["compound_name"] + "\",", match[1]["match_factor"], match[0], match[2])

In [ ]:
# OPTIONAL : ACP with all pixels 
features = np.array([chroma.flatten() for chroma in aligned_chromatos])
features.shape
features_new_cd, pca = find_biom.acp(features, n_components=2)
plot.plot_acp(features_new_cd, labels)

In [ ]:
# find discrminant pixels
discriminant_pixels = discriminant_pixel.find_discriminant_pixels((chromato, time_rn, None), aligned_chromatos, vij_posi_vs_other, disp=False, max_pixel=500, local_max_filter=True)

In [ ]:
# retrieve intensities associated with disc pixels in each chromatogram
features = np.array([chroma[discriminant_pixels[:, 0], discriminant_pixels[: , 1]] for chroma in aligned_chromatos])
features.shape
norm = False
if norm:
    features = normalize(features, norm='l1')
features_new_cd_disc_pix, pca_disc_pix = find_biom.acp(features, n_components=2)
# ACP with disc pixels
plot.plot_acp(features_new_cd_disc_pix, labels)

You may find more details for this approach in results/discriminant_pixel_aligned_chroma.ipynb